In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.display import Image as DisplayImage
from PIL import Image as PILImage
from fastai.vision import *
from fastai.vision.gan import *
import torch.nn as nn
import torch.nn.functional as F

You should set the following option to True if the notebook isn't located in the file system inside a clone of the git repo (with the needed Python modules available) it belongs to; i.e., it's running independently.

In [ ]:
run_as_standalone_nb = True

In [ ]:
# This cell needs to be executed before importing local project modules, like import core.gan
if run_as_standalone_nb:
    root_lib_path = os.path.abspath('generative-lab')
    if not os.path.exists(root_lib_path):
        !git clone https://github.com/davidleonfdez/generative-lab.git
    if root_lib_path not in sys.path:
        sys.path.insert(0, root_lib_path)
else:
    import local_lib_import

In [ ]:
# Local project modules. Must be imported after local_lib_import or cloning git repo.
from core.gan import CustomGANLearner, load_gan_learner, save_gan_learner, train_checkpoint_gan
from core.net_builders import pseudo_res_critic, pseudo_res_generator

In [ ]:
img_size = 64
img_n_channels = 3
batch_size = 64

# DATA

In [ ]:
ds_url = "http://vis-www.cs.umass.edu/lfw/lfw"

In [ ]:
realImagesPath = untar_data(ds_url)
realImagesPath

In [ ]:
sampleImg1Path = realImagesPath/'Aaron_Eckhart/Aaron_Eckhart_0001.jpg'

In [ ]:
im = PILImage.open(sampleImg1Path)
im.size

In [ ]:
DisplayImage(filename=str(sampleImg1Path))

In [ ]:
def get_data(path, bs, size, noise_sz=100):
    return (GANItemList.from_folder(path, noise_sz=noise_sz)
               .split_none()
               .label_from_func(noop)
               .transform(tfms=[[crop_pad(size=size, row_pct=(0,1), col_pct=(0,1))], []], size=size, tfm_y=True)
               .databunch(bs=bs)
               .normalize(stats = [torch.tensor([0.5,0.5,0.5]), torch.tensor([0.5,0.5,0.5])], do_x=False, do_y=True))

In [ ]:
data = get_data(realImagesPath, batch_size, img_size)
data.show_batch()

# GENERATOR

Let's create the generator with the helper method that fastai provides.

Input is bs x noise_sz * 1 * 1<br>
**n_features** is the number of feature maps (so kernels) generated after penultimate layer (the last layer of course outputs n_channels) if n_extra_layers = 0 . At the beginning there will be n_features * 2^(n_intermediate_convtrans_blocks), and this number will be reduced by half in any subsequent layer.

`basic_generator(in_size:int, n_channels:int, noise_sz:int=100, n_features:int=64, n_extra_layers=0, **conv_kwargs)`

In [ ]:
generator = basic_generator(img_size, img_n_channels)

# CRITIC

Let's create the discriminator with the helper method that fastai provides.<br>

**n_features** is the number of feature maps (so kernels) generated after first layer (from the n_channels of the input). This number will be doubled in any subsequent layer.

`basic_critic(in_size:int, n_channels:int, n_features:int=64, n_extra_layers:int=0, **conv_kwargs)`

In [ ]:
critic = basic_critic(img_size, img_n_channels)

# GAN LEARNER

In [ ]:
learner = CustomGANLearner.wgan(data, generator, critic)

# TRAINING

## First attempt: default hyperparameters (wd=0.01, Adam(beta1=0.9,beta2=0.99)), lr=1e-4

In [ ]:
lr = 1e-4
learner.fit(50, lr)

In [ ]:
learner.fit(50, lr)

In [ ]:
save_gan_learner(learner, 'wganTr1_100ep.pth')

## Second attempt: use fastai recommended hyperparameters

In [ ]:
lr = 1e-4
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
critic = basic_critic(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner.wgan(data, generator, critic, opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)
learner.fit(50, lr)

In [ ]:
learner.fit(50, lr)

In [ ]:
save_gan_learner(learner, 'wganTr2_100ep.pth')

In [ ]:
learner.show_results(ds_type=DatasetType.Train)

## Third attempt: increment batch size

In [ ]:
lr = 2e-4
data = get_data(realImagesPath, 128, img_size)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
critic = basic_critic(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner.wgan(data, generator, critic, switch_eval=False, 
                                opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(50, lr)

In [ ]:
learner.fit(50, lr)

In [ ]:
save_gan_learner(learner, 'wganTr3_100ep.pth')

In [ ]:
learner.show_results(ds_type=DatasetType.Train)

In [ ]:
train_checkpoint_gan(learner, n_epochs=400, initial_epoch=100, filename_start='wganTr3_', lr=lr)

In [ ]:
learner.show_results(ds_type=DatasetType.Train)

## Fourth attempt: use standard Adam parameters

In [ ]:
lr = 2e-4
data = get_data(realImagesPath, 128, img_size)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
critic = basic_critic(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner.wgan(data, generator, critic, switch_eval=False, opt_func = partial(optim.Adam, betas = (0.9,0.999)), wd=0.)
learner.fit(50, lr)
save_gan_learner(learner, 'wganTr4_50ep.pth')

## Fifth attempt: smaller noise vector

In [ ]:
noise_sz = 60
lr = 2e-4
data = get_data(realImagesPath, batch_size, img_size, noise_sz=noise_sz)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1, noise_sz=noise_sz)
critic = basic_critic(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner.wgan(data, generator, critic, switch_eval=False, opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)
learner.fit(50, lr)
save_gan_learner(learner, 'wganTr5_50ep.pth')

In [ ]:
# Leave data as it was
data = get_data(realImagesPath, batch_size, img_size)

## Sixth attempt: add residual blocks

### 6.1: Add residual blocks in critic and generator

In [ ]:
lr = 2e-4
generator = pseudo_res_generator(img_size, img_n_channels)
critic = pseudo_res_critic(img_size, img_n_channels)
learner = CustomGANLearner.wgan(data, generator, critic, switch_eval=False, 
                                opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(50, lr)

In [ ]:
learner.fit(50, lr)

In [ ]:
save_gan_learner(learner, 'wganTr6_1_100ep.pth')

### 6.2: Using bigger batch size...

In [ ]:
data = get_data(realImagesPath, 128, img_size)
generator = pseudo_res_generator(img_size, img_n_channels)
critic = pseudo_res_critic(img_size, img_n_channels)
learner = CustomGANLearner.wgan(data, generator, critic, switch_eval=False, 
                                opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(50, lr)

In [ ]:
learner.fit(50, lr)

In [ ]:
save_gan_learner(learner, 'wganTr6_2_100ep.pth')

#### 6.2.2 With two extra layers

Theoretically extra layers at the end should give better results (or not worse) if residual, which is not the case right now, maybe could try it later.

In [ ]:
lr = 2e-4
data = get_data(realImagesPath, 128, img_size)
generator = pseudo_res_generator(img_size, img_n_channels, n_extra_layers=2)
critic = pseudo_res_critic(img_size, img_n_channels, n_extra_layers=2)
learner = CustomGANLearner.wgan(data, generator, critic, switch_eval=False, 
                                opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(50, lr)

In [ ]:
learner.fit(50, lr)

In [ ]:
save_gan_learner(learner, 'wganTr6_2_2_100ep.pth')

#### 6.2.3 With one extra layer

In [ ]:
data = get_data(realImagesPath, 128, img_size)
generator = pseudo_res_generator(img_size, img_n_channels, n_extra_layers=1)
critic = pseudo_res_critic(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner.wgan(data, generator, critic, switch_eval=False, 
                                opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(50, lr)

In [ ]:
learner.fit(50, lr)

In [ ]:
save_gan_learner(learner, 'wganTr6_2_3_100ep.pth')

#### 6.2.4: Switch the order of residual and conv blocks in critic

In [ ]:
lr = 2e-4
data = get_data(realImagesPath, 128, img_size)
generator = pseudo_res_generator(img_size, img_n_channels, n_extra_layers=1)
critic = pseudo_res_critic(img_size, img_n_channels, n_extra_layers=1, conv_before_res=False)
learner = CustomGANLearner.wgan(data, generator, critic, switch_eval=False, 
                                opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(50, lr)

In [ ]:
learner.fit(50, lr)

In [ ]:
save_gan_learner(learner, 'wganTr6_2_4_100ep.pth')

#### 6.2.5 With bigger learning rate...

In [ ]:
lr = 1e-3
data = get_data(realImagesPath, 128, img_size)
generator = pseudo_res_generator(img_size, img_n_channels, n_extra_layers=1)
critic = pseudo_res_critic(img_size, img_n_channels, n_extra_layers=1, conv_before_res=False)
learner = CustomGANLearner.wgan(data, generator, critic, switch_eval=False, 
                                opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(50, lr)

In [ ]:
learner.fit(50, lr)

In [ ]:
save_gan_learner(learner, 'wganTr6_2_5_100ep.pth')

In [ ]:
learner.show_results(ds_type=DatasetType.Train)

#### 6.2.6 With not so big lr...

In [ ]:
lr = 5e-4
data = get_data(realImagesPath, 128, img_size)
generator = pseudo_res_generator(img_size, img_n_channels, n_extra_layers=1)
critic = pseudo_res_critic(img_size, img_n_channels, n_extra_layers=1, conv_before_res=False)
learner = CustomGANLearner.wgan(data, generator, critic, switch_eval=False, 
                                opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(50, lr)

In [ ]:
learner.fit(50, lr)

In [ ]:
save_gan_learner(learner, 'wganTr6_2_6_100ep.pth')

In [ ]:
learner.show_results(ds_type=DatasetType.Train)

### 6.3: Use residual blocks only in critic

In [ ]:
lr = 2e-4
data = get_data(realImagesPath, 128, img_size)
generator = basic_generator(img_size, img_n_channels, n_extra_layers=1)
critic = pseudo_res_critic(img_size, img_n_channels, n_extra_layers=1)
learner = GANLearner.wgan(data, generator, critic, switch_eval=False, 
                          opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(50, lr)

In [ ]:
learner.fit(50, lr)

In [ ]:
save_gan_learner(learner, 'wganTr6_3_100ep.pth')

### 6.4: Use residual blocks only in generator

In [ ]:
lr = 2e-4
data = get_data(realImagesPath, 128, img_size)
generator = pseudo_res_generator(img_size, img_n_channels, n_extra_layers=1)
critic = basic_critic(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner.wgan(data, generator, critic, switch_eval=False, 
                                opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(50, lr)

In [ ]:
learner.fit(50, lr)

In [ ]:
save_gan_learner(learner, 'wganTr6_4_100ep.pth')

In [ ]:
learner.show_results(ds_type=DatasetType.Train)

## Attempt 7: add dense blocks

### 7.1: Dense both critic and generator

In [ ]:
lr = 2e-4
data = get_data(realImagesPath, 128, img_size)
generator = pseudo_res_generator(img_size, img_n_channels, n_extra_layers=1, dense=True)
critic = pseudo_res_critic(img_size, img_n_channels, n_extra_layers=1, dense=True)
learner = CustomGANLearner.wgan(data, generator, critic, switch_eval=False, 
                                opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)

In [ ]:
learner.fit(30, lr)

In [ ]:
save_gan_learner(learner, 'wganTr7_1_60ep.pth')

### 7.2: Dense generator only

In [ ]:
lr = 2e-4
data = get_data(realImagesPath, 128, img_size)
generator = pseudo_res_generator(img_size, img_n_channels, n_extra_layers=1, dense=True)
critic = basic_critic(img_size, img_n_channels, n_extra_layers=1)
learner = CustomGANLearner.wgan(data, generator, critic, switch_eval=False, 
                                opt_func = partial(optim.Adam, betas = (0.,0.99)), wd=0.)

In [ ]:
learner.fit(30, lr)

In [ ]:
save_gan_learner(learner, 'wganTr7_2_30ep.pth')

In [ ]:
learner.fit(30, lr)

In [ ]:
save_gan_learner(learner, 'wganTr7_2_60ep.pth')

In [ ]:
learner.show_results(ds_type=DatasetType.Train)